In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, row_number
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.window import Window


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1656829944622_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
spark.conf.set("spark.sql.parquet.compression.codec", "gzip")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
input_data = "s3://udacity-dend/"
output_data = "s3://udacity-project-datalake/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# get filepath to song data file
song_data = os.path.join(input_data, 'song_data', 'A', 'A', 'A')
df = spark.read.json(song_data)
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [19]:
# extract columns to create songs table
songs_table = df.select(['song_id', 'title', 'artist_id', 'year', 'duration'])

# write songs table to parquet files partitioned by year and artist
songs_table.write.parquet(os.path.join(output_data, 'songs'), 'overwrite', partitionBy=['year', 'artist_id'])

# show sample data
songs_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOAFBCP12A8C13CC7D|King Of Scurf (20...|ARTC1LV1187B9A4858|1972|301.40036|
|SOKTJDS12AF72A25E5|Drown In My Own T...|ARA23XO1187B9AF18F|   0|  192.522|
|SOEKAZG12AB018837E|I'll Slap Your Fa...|ARSVTNL1187B992A91|2001|129.85424|
|SOQPWCR12A6D4FB2A3|A Poor Recipe For...|AR73AIO1187B9AD57B|2005|118.07302|
|SOBRKGM12A8C139EF6|Welcome to the Pl...|ARXQBR11187B98A2CC|1985|821.05424|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows

In [20]:
# extract columns to create artists table
columns = ['artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
columns = [col + ' as ' + col.replace('artist_', '') for col in columns]
artists_table = df.selectExpr('artist_id', *columns)

# write artists table to parquet files
artists_table.write.parquet(os.path.join(output_data, 'artists'), 'overwrite')

# show sample data
artists_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+--------+----------+
|         artist_id|                name|            location|latitude| longitude|
+------------------+--------------------+--------------------+--------+----------+
|ARTC1LV1187B9A4858|  The Bonzo Dog Band|Goldsmith's Colle...| 51.4536|  -0.01802|
|ARA23XO1187B9AF18F|     The Smithereens|Carteret, New Jersey|40.57885| -74.21956|
|ARSVTNL1187B992A91|       Jonathan King|     London, England|51.50632|  -0.12714|
|AR73AIO1187B9AD57B|   Western Addiction|   San Francisco, CA|37.77916|-122.42005|
|ARXQBR11187B98A2CC|Frankie Goes To H...|  Liverpool, England|    null|      null|
+------------------+--------------------+--------------------+--------+----------+
only showing top 5 rows

In [21]:
songs_table.printSchema()
artists_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- duration: double (nullable = true)

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

In [28]:
# get filepath to log data file
log_data = input_data + 'log_data/*/*/'
# read log data file
df = df = spark.read.json(log_data)
# filter by actions for song plays
df = df.filter(df.page == 'NextSong').dropDuplicates()
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [23]:
 # extract columns for users table    
users_table = df.select('userId', 'firstName', 'lastName', 'gender', 'level') \
                .dropDuplicates()
    
# write users table to parquet files
users_table.write.parquet(output_data  + 'users/' + 'users.parquet', 'overwrite')

# show sample data
users_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    80|    Tegan|  Levine|     F| paid|
|    48|   Marina|  Sutton|     F| free|
|    16|    Rylan|  George|     M| paid|
|    83|  Stefany|   White|     F| free|
|     8|   Kaylee| Summers|     F| free|
+------+---------+--------+------+-----+
only showing top 5 rows

In [24]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x:  datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
df_time_cvt = df.withColumn('timestamp', get_timestamp(df.ts))

# create datetime column from original timestamp column
get_datetime = udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
df_time_cvt = df_time_cvt.withColumn("datetime", get_datetime(df.ts))

# extract columns to create time table
time_table = df_time_cvt.select(
    col('datetime').alias('start_time'),
    hour('datetime').alias('hour'),
    dayofmonth('datetime').alias('day'),
    month('datetime').alias('month'),
    year('datetime').alias('year'),
    weekofyear('datetime').alias('week')
).dropDuplicates()

# write time table to parquet files partitioned by year and month
time_table.write.parquet(output_data + 'time/' + 'time.parquet', 'overwrite', partitionBy=['year','month'])

# show sample data
time_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+---+-----+----+----+
|         start_time|hour|day|month|year|week|
+-------------------+----+---+-----+----+----+
|2018-11-15 21:54:05|  21| 15|   11|2018|  46|
|2018-11-05 04:45:12|   4|  5|   11|2018|  45|
|2018-11-09 23:24:09|  23|  9|   11|2018|  45|
|2018-11-15 11:25:06|  11| 15|   11|2018|  46|
|2018-11-23 02:35:36|   2| 23|   11|2018|  47|
+-------------------+----+---+-----+----+----+
only showing top 5 rows

In [25]:
# read in song data to use for songplays table
song_df = spark.read.json(input_data + 'song_data/A/A/A/*.json')

# extract columns from joined song and log datasets to create songplays table 
songplays_table = df_time_cvt.join(
    song_df,

    # because I only take small subset, so I have to use this condition, it's not correct but it has data output
    (df.song == song_df.title) | (df.artist == song_df.artist_name), 

    # when you run full dataset, you can use this condition
    # (df_time_cvt.song == song_df.title) & (df_time_cvt.artist == song_df.artist_name) & (df_time_cvt.length == song_df.duration),
    how='inner')

songplays_table = songplays_table.withColumn("songplay_id", row_number().over(Window.partitionBy('timestamp').orderBy("timestamp")))

songplays_table = songplays_table.select(
    "songplay_id", "timestamp", "userId",
    "level", "song_id", "artist_id",
    "sessionId", "location", "userAgent",
    month('timestamp').alias('month'),
    year('timestamp').alias('year')) \
        .withColumnRenamed("userId","user_id")        \
        .withColumnRenamed("timestamp","start_time")  \
        .withColumnRenamed("sessionId","session_id")  \
        .withColumnRenamed("userAgent", "user_agent")

songplays_table = songplays_table.dropDuplicates()

# write songplays table to parquet files partitioned by year and month
songplays_table.write.parquet(output_data + 'songplays/' + 'songplays.parquet', 'overwrite', partitionBy=['year','month'])

# show sample data
songplays_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----+----+
|songplay_id|         start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|month|year|
+-----------+-------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----+----+
|          1|2018-11-06 20:05:45|     97| paid|SOBLFFE12AF72AA5BA|ARJNIUY12298900C91|       293|Lansing-East Lans...|"Mozilla/5.0 (X11...|   11|2018|
|          1|2018-11-27 17:08:42|     36| paid|SORRNOC12AB017F52B|ARSZ7L31187FB4E610|       957|Janesville-Beloit...|"Mozilla/5.0 (Win...|   11|2018|
|          1|2018-11-20 00:51:41|     25| paid|SONRWUU12AF72A4283|ARGE7G11187FB37E05|       594|    Marinette, WI-MI|"Mozilla/5.0 (Win...|   11|2018|
|          1|2018-11-28 16:54:51|     14| free|SOIGHOD12A8C13B5A1|ARY589G1187B9A9F4E|       929|    

In [26]:
users_table.printSchema()
time_table.printSchema()
songplays_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

root
 |-- userId: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

root
 |-- start_time: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (